In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt 
import os


In [ ]:
! unzip -q /kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip


In [ ]:
! unzip -q /kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [ ]:
datasets_train = os.listdir('train')
datasets_test = os.listdir('test')


In [ ]:
datasets_train[0:10]  # data are images with this name  ok
datasets_test[0:5]


In [ ]:
labels = [] 
for imagename in datasets_train:
    if 'dog' in  imagename:
        labels.append('dog')
    elif 'cat' in imagename:
        labels.append('cat')


In [ ]:
#preparing a dataframe of images and their labels
dfx = pd.DataFrame()
dfx['imagename']= datasets_train
dfx['labels']= labels
dftest = pd.DataFrame()
dftest['image'] = datasets_test


In [ ]:
dftest.head()
dfx.head()


In [ ]:
def show_image(imageadd):
    image = cv2.imread(imageadd)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.title('imagename')
    plt.imshow(image)
show_image('/kaggle/working/train/'+ dfx.imagename[0])


In [ ]:
plt.figure(figsize=(20,20))
for i in range(10):
    image = cv2.imread('/kaggle/working/train/'+ dfx.loc[i,'imagename'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.subplot(2,5,i+1)
    plt.title(dfx.loc[i,'imagename'])
    plt.imshow(image)


In [ ]:
#checking balance data
dfx.describe()
dfx.value_counts('labels')
dfx.duplicated('imagename').sum()


In [ ]:
images = []
for imagename in os.listdir('train'):
    
    image = cv2.imread('/kaggle/working/train/'+ imagename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images.append(image)
    


In [ ]:
len(images)


In [ ]:
x = np.array(images)
listshape=[]

for i in range(len(x)):
    listshape.append(x[i].shape)
shapearray = np.array(listshape)
shapearray[0:5]


In [ ]:
#mean pixel value od all images
shapearray.mean(axis=0)


In [ ]:
#checking if all images are 3 layer rbg
count = 0
for i in range(len(shapearray)):
    if shapearray[i,2] !=3:
        count = count+1
count


In [ ]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,
#                                                       validation_split= 0.1,#
                                                       preprocessing_function=tf.keras.applications.vgg16.preprocess_input ,
                                                      width_shift_range=0.1,
                                                      height_shift_range=0.1,
                                                      zoom_range=0.2,
                                                      rotation_range=25,
                                                     )


In [ ]:
os.mkdir('viewaugimages')


In [ ]:
dfx.imagename[8]


In [ ]:
image = dfx.imagename[8]
image = cv2.imread('/kaggle/working/train/'+ image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image2 = np.expand_dims(image,axis=0)
image2.shape


In [ ]:
#checking images augmented by idg
i=0
for _ in idg.flow(image2,save_to_dir='viewaugimages'):
    i=i+1
    if i >24:
        break


In [ ]:
auglist=[]
for imagename in os.listdir('viewaugimages'):
    image = cv2.imread('viewaugimages/'+ imagename)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    auglist.append(image)


In [ ]:
len(auglist)


In [ ]:
plt.figure(figsize= (20,20))
for i in range(25):
    image = auglist[i]
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.subplot(5,5,i+1)
    plt.imshow(image)
    


In [ ]:
bs = 32


In [ ]:
train_idg = idg.flow_from_dataframe(dfx,directory = '/kaggle/working/train/',
                                    x_col = 'imagename',y_col = 'labels',
                                    target_size =(180,200),
                                    batch_size =bs,
                                    subset = 'training')


In [ ]:
# valid_idg = idg.flow_from_dataframe(dfx,directory = '/kaggle/working/train/',
#                                     x_col = 'imagename',y_col = 'labels',
#                                     target_size =(180,200),
#                                     batch_size =bs,
#                                     subset = 'validation')


In [ ]:
VGG16transfer = tf.keras.applications.VGG16(
    include_top=False,
    input_shape= (180,200,3),
    weights="imagenet"
    
)
#look at model structure
tf.keras.utils.plot_model(VGG16transfer)
#making weights not trainable of vgg
for layer in VGG16transfer.layers:
    print(layer.name)
    print(layer.input_shape)
    print(layer.output_shape)

    print(layer.trainable)
    layer.trainable = False
    print(layer.trainable)
    


In [ ]:
flat1 = tf.keras.layers.Flatten() (VGG16transfer.output)
d1 = tf.keras.layers.Dense(32,activation = 'relu') (flat1)
pred = tf.keras.layers.Dense(2,activation = 'softmax') (d1)


model1 = tf.keras.Model(inputs = [VGG16transfer.input], outputs = [pred])

for layer in model1.layers:
    print(layer.trainable)


In [ ]:
model1.summary()


In [ ]:
model1.compile(optimizer = tf.keras.optimizers.SGD(learning_rate = 0.0031415) ,
              loss = tf.keras.losses.categorical_crossentropy ,
              metrics=['acc']
             )


In [ ]:
model1.fit(train_idg, epochs = 1,batch_size= bs)


In [ ]:
dict = model1.history.history


In [ ]:
plt.figure(figsize=(15,10))
plt.plot(dict['acc'])
# plt.plot(dict['val_acc'])
plt.show()


In [ ]:
plt.figure(figsize=(15,10))
plt.plot(dict.get('loss'))
# plt.plot(dict.get('val_loss')
plt.show()


In [ ]:
dftest.sample(2)


In [ ]:
idg2 = tf.keras.preprocessing.image.ImageDataGenerator(
                                                         preprocessing_function=tf.keras.applications.vgg16.preprocess_input
                                                           )
test_gen = idg2.flow_from_dataframe(
    dftest, 
    '/kaggle/working/test', 
    x_col='image',
    class_mode= None,
    target_size=(180,200),
    batch_size=bs,
    shuffle=False
)


In [ ]:
predict = model1.predict(test_gen,batch_size=bs,max_queue_size=1, verbose = 1)


In [ ]:
p=4
print((predict[4,1]))#prob of being a dog
show_image('/kaggle/working/test/'+ dftest.image[p])


In [ ]:
labels_test = (predict[:,1])
labels_test[0:5]
# dftest.head()


In [ ]:
result = pd.DataFrame()
result['id']=   [i for i in range(1,len(dftest)+1)]
result['label']=labels_test
result.head()


In [ ]:
#saving to  sumbission.csv


In [ ]:
result.to_csv('submission.csv',index = False)
